In [15]:
from Assignment3_part1 import *
from tqdm import tqdm

## Variables qui nous intéressent

* SAA_weights_IS
* SAA_weights_OS
* TAA_weights_IS
* TAA_weights_OS
* weight_target_IS
* weight_target_OS
* benchmark_weights_IS
* benchmark_weights_OS

### Benchmark (50% Equities / 50% Bond)

### Return

* returns_IS_price
* returns_OS_price

In [19]:
# SAA
SAA_active_weight_IS = SAA_weights_IS - benchmark_weights_IS
SAA_active_weight_OS = SAA_weights_OS - benchmark_weights_OS

# TAA
TAA_active_weight_IS = TAA_weights_IS - benchmark_weights_IS
TAA_active_weight_OS = TAA_weights_OS - benchmark_weights_OS

# Real portfolio
active_weight_target_IS = weight_target_IS - benchmark_weights_IS
active_weight_target_OS = weight_target_OS - benchmark_weights_OS

In [9]:
MCR_SAA_active_IS = MCR_calc(SAA_active_weight_IS,returns_IS_price)

MCR_TAA_active_IS = MCR_calc(TAA_active_weight_IS,returns_IS_price)

MCR_real_IS = MCR_calc(active_weight_target_IS,returns_IS_price)

In [28]:
# SAA
MCR_SAA = [MCR_SAA_active_IS]
MCR_TAA = [MCR_TAA_active_IS]
MCR_real = [MCR_real_IS]

SAA_rolling = SAA_active_weight_IS.copy()
TAA_rolling = TAA_active_weight_IS.copy()
Real_rolling = weight_target_IS.copy()
return_rolling = returns_IS_price.copy()
for i in tqdm(range(len(SAA_active_weight_OS))):
    SAA_rolling = SAA_rolling.append(SAA_active_weight_OS.iloc[i,:])
    TAA_rolling = TAA_rolling.append(TAA_active_weight_OS.iloc[i,:])
    Real_rolling = Real_rolling.append(active_weight_target_OS.iloc[i,:])
    return_rolling = return_rolling.append(returns_OS_price.iloc[i,:])
    MCR_SAA.append(MCR_calc(SAA_rolling,return_rolling))
    MCR_TAA.append(MCR_calc(TAA_rolling,return_rolling))
MCR

100%|██████████| 124/124 [00:01<00:00, 89.21it/s]


.044131   -0.044131   -0.044131   -0.044131  ...   -0.044131   
 1        0.003508    0.003508    0.003508    0.003508  ...    0.003508   
 2       -0.002583   -0.002583   -0.002583   -0.002583  ...   -0.002583   
 3       -0.017459   -0.017459   -0.017459   -0.017459  ...   -0.017459   
 4        0.001843    0.001843    0.001843    0.001843  ...    0.001843   
 5       -0.029972   -0.029972   -0.029972   -0.029972  ...   -0.029972   
 6       -0.031851   -0.031851   -0.031851   -0.031851  ...   -0.031851   
 
 Dates  2019-12-31  2020-01-31  2020-02-28  2020-03-31  2020-04-30  2020-05-29  \
 0       -0.044131   -0.044131   -0.044131   -0.044131   -0.044131   -0.044131   
 1        0.003508    0.003508    0.003508    0.003508    0.003508    0.003508   
 2       -0.002583   -0.002583   -0.002583   -0.002583   -0.002583   -0.002583   
 3       -0.017459   -0.017459   -0.017459   -0.017459   -0.017459   -0.017459   
 4        0.001843    0.001843    0.001843    0.001843    0.001843    0.00

In [29]:
MCR[-1]

Dates,2000-10-31,2000-11-30,2000-12-29,2001-01-31,2001-02-28,2001-03-30,2001-04-30,2001-05-31,2001-06-29,2001-07-31,...,2020-07-31,2020-08-31,2020-09-30,2020-10-30,2020-11-30,2020-12-31,2021-01-29,2021-02-26,2021-03-31,2021-04-30
0,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,...,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338,-0.044338
1,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,...,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501,0.003501
2,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,...,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630,-0.002630
3,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,...,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222,-0.017222
4,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,...,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353,0.002353
5,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,...,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283,-0.030283
6,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,...,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247,-0.032247
